Bayu Kurniawan
21/480093/GE/09662

IMPORT LIBRARY

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.nn.modules.loss import BCEWithLogitsLoss
import rasterio
import numpy as np
import matplotlib
import os
import glob
import shutil
import matplotlib.pyplot as plt
import cv2
from IPython.core.pylabtools import figsize
from Model.load_data import CustomGeoDataset

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("[INFO] training using {} ...".format(DEVICE))

[INFO] training using cuda ...


LOAD MODEL YANG SUDAH DITRAIN

In [2]:
#load model
modelunet = torch.load('modelunet.pth')
modelunet.eval()

unet(
  (conv_block1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv_block11): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv_block22): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )

LOAD DATA DAN INISIASI FOLDER BARU UNTUK OUTPUT HASILNYA

In [6]:
#panggil data
split_folder = "./split/"
split_image = glob.glob(os.path.join(split_folder, "*.tif"))
print(len(split_image))

#output folder (buat dulu sebelum dirunning)
outfolder = './Hasil'

4250


COBA DITERAPKAN UNTUK SATU IMAGE

In [7]:
#testing object detection untuk satu image

import torchvision.transforms as F

t1 = split_image[0]
filename = os.path.basename(t1)
print(filename)

#buka data
t2 = rasterio.open(t1)
t3 = t2.read()
t3 = np.expand_dims(t3, axis = 0)
print(t3.shape)

#data dikonversi ke 0-1 dengan membagi menjadi nilai maksimal citra, fungsi ini didefinisikan pada script load data sehingga perlu diterapkan
t3 = t3/65535

# Convert menjadi float32 karena pytorch butuh input dalam float
t3_flts = t3.astype(np.float32)

# Convert menjadi torch tensor, simpan dalam device yang digunakan dalam proses komputasi (cpu/gpu)
t3_tensor = torch.from_numpy(t3_flts).to(DEVICE)

# klasifikasi dengan model alexnet
t3_pred = modelunet(t3_tensor)
t3_pred = torch.sigmoid(t3_pred) # diperlukan saat klasifikasi binary untuk mengubah hasil prediksi menjadi probablitas dengan nilai 0 - 1
t3_pred = t3_pred.cpu().detach().numpy()
t3_prob = t3_pred[0]
print(t3_prob)

#kelaskan menjadi 0 atau 1 (binary mapping)
threshold = 0.5
t3_class = t3_prob
t3_class[t3_class >= threshold] = 1
t3_class[t3_class < threshold] = 0
print(t3_class.shape)

#ubah menjadi raster
out_meta = t2.meta
out_meta.update({'driver':'GTiff', 'height':128, 'width':128, 'count':1, 'crs': t2.crs, 'transforms': t2.transform})
with rasterio.open('hasil_unet_{}'.format(filename), 'w', **out_meta) as object:
    object.write(t3_class)



crop_0_0.tif
(1, 1, 128, 128)
[[[0.01533674 0.00252063 0.00057081 ... 0.00163745 0.00401371 0.01751622]
  [0.00354096 0.0004662  0.00013727 ... 0.00041717 0.00077098 0.00360758]
  [0.00196403 0.0003697  0.00026022 ... 0.00044457 0.00043658 0.00105631]
  ...
  [0.01424167 0.00267464 0.00082833 ... 0.00841612 0.00244411 0.00401742]
  [0.02864383 0.0073679  0.00114474 ... 0.00840826 0.00294804 0.00781961]
  [0.05669362 0.01396277 0.00357007 ... 0.01153932 0.01105988 0.03172388]]]
(1, 128, 128)


LOOPING CNN UNET PADA CITRA

In [8]:
#loop cnn unet (1: 'path1')
for i, obj in enumerate(split_image):
    t1 = obj
    filename = os.path.basename(t1)
    print(filename)

    #buka data
    t2 = rasterio.open(t1)
    t3 = t2.read() #3D (C, H, W)
    t3 = np.expand_dims(t3, axis = 0)  #4D (BatcSize, C, H, W)
    print(t3.shape)

    #data dikonversi ke 0-1 dengan membagi menjadi nilai maksimal citra, fungsi ini didefinisikan pada script load data sehingga perlu diterapkan
    t3 = t3/65535

    # Convert menjadi float32 karena pytorch butuh input dalam float
    t3_flts = t3.astype(np.float32)

    # Convert menjadi torch tensor, simpan dalam device yang digunakan dalam proses komputasi (cpu/gpu)
    t3_tensor = torch.from_numpy(t3_flts).to(DEVICE)

    # klasifikasi dengan model unet
    t3_pred = modelunet(t3_tensor)
    t3_pred = torch.sigmoid(t3_pred) # diperlukan saat klasifikasi binary untuk mengubah hasil prediksi menjadi probablitas dengan nilai 0 - 1
    t3_pred = t3_pred.cpu().detach().numpy()
    #t3_pred = t3_pred.detach().numpy()
    t3_prob = t3_pred[0]
    #print(t3_prob)

    #kelaskan menjadi 0 atau 1 (binary mapping)
    threshold = 0.5
    t3_class = t3_prob
    t3_class[t3_class >= threshold] = 1
    t3_class[t3_class < threshold] = 0
    print(t3_class.shape)

    #ubah menjadi raster
    out_meta = t2.meta
    out_meta.update({'driver':'GTiff', 'height':128, 'width':128, 'count':1, 'crs': t2.crs, 'transforms': t2.transform})
    with rasterio.open(os.path.join(outfolder, 'hasil_unet_{}'.format(filename)), 'w', **out_meta) as object:
        object.write(t3_class)



crop_0_0.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1024.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1056.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1088.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1120.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1152.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1184.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1216.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1248.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_128.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1280.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1312.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1344.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1376.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1408.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1440.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1472.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1504.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1536.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1568.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_160.tif
(1, 1, 128, 128)
(1, 128, 128)
crop_0_1600.tif
(1

MOSAIK POTONGAN-POTONGAN GAMBAR HASIL TRAINING CNN UNET 

In [9]:
#mosaic hasil prediksi
from rasterio.merge import merge, copy_sum
from rasterio.plot import show
import matplotlib.pyplot as plt

list_hasil = glob.glob(os.path.join(outfolder, "*tif"))
print(len(list_hasil))

# Open the files as datasets
src_files_to_mosaic = []
for fp in list_hasil:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

# Merge the datasets
mosaic, out_trans = merge(src_files_to_mosaic, method = copy_sum)

# Update the metadata with the new dimensions, transform, and crs
out_meta = src.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans
})

# Save the merged result as a new GeoTIFF file
with rasterio.open("unet2_classified_mosaic.tif", "w", **out_meta) as dest:
    dest.write(mosaic)

# Close the datasets
for src in src_files_to_mosaic:
    src.close()


4250
